In [2]:
import glob
import os
import pathlib

import numpy as np
import pandas as pd

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import functional as F

from torchvision import transforms
from torchinfo import torchinfo
from tqdm import tqdm
import matplotlib.pyplot as plt

import albumentations as A
import torch.nn.functional as F

from PIL import Image

import torchmetrics
from torchvision.utils import save_image, make_grid
import cv2
import os, glob

import efficientunet
import random

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

c:\ProgramData\Anaconda3\envs\cvproject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset_path = 'Datasets/Mask detection/Face Mask Dataset/'
train_dir = dataset_path+'Train/'
val_dir = dataset_path+'Validation/'
test_dir = dataset_path+'Test/'

In [9]:
sizes = (256, 256)            # before (64, 64)

rescale_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(sizes, antialias= False)
])

In [10]:
torch.hstack([torch.ones(5), torch.zeros(4)])

tensor([1., 1., 1., 1., 1., 0., 0., 0., 0.])

In [11]:
def get_files(path):
    mask_files = glob.glob(path + 'WithMask/' +'*.png')
    nomask_files = glob.glob(path + 'WithoutMask/' +'*.png')

    mask_images = [rescale_transform(Image.open(x)) for x in tqdm(mask_files)]
    unmasked_images = [rescale_transform(Image.open(x)) for x in tqdm(nomask_files)]

    mask_labels = torch.ones(len(mask_images))
    unmask_labels = torch.zeros(len(unmasked_images))

    mask_images = torch.stack(mask_images)
    unmasked_images = torch.stack(unmasked_images)
    images = torch.vstack([mask_images, unmasked_images])
    labels = torch.hstack([mask_labels, unmask_labels])

    return images, labels

In [12]:
train_images, train_labels = get_files(train_dir)

100%|██████████| 5000/5000 [00:06<00:00, 828.29it/s]


In [13]:
val_images, val_labels = get_files(val_dir)

100%|██████████| 400/400 [00:00<00:00, 820.37it/s]


In [14]:
normalisation = transforms.Normalize(0.5, 0.5)

augmentation = transforms.RandomAffine(
    30, (0.15, 0.15), (0.8, 1.2), 10
)

In [15]:
class MaskDataset(Dataset):
    def __init__(self, images, labels, augmentation = None):
        self.images = images
        self.augmentation = augmentation
        self.labels = labels

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = normalisation(self.images[idx])
        if(self.augmentation is not None):
            img = self.augmentation(img)
        return img, self.labels[idx]

In [16]:
train_dataset = MaskDataset(train_images, train_labels, augmentation = augmentation)
val_dataset = MaskDataset(val_images, val_labels)

In [17]:
BATCH_SIZE = 250

train_dataloader = DataLoader(train_dataset, BATCH_SIZE, shuffle = True)
val_dataloader   = DataLoader(val_dataset, BATCH_SIZE, shuffle = True)

In [44]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 5, 3)        # out = 254
        self.conv2 = nn.Conv2d(5, 10, 3)       # in = 127, out = 125
        self.conv3 = nn.Conv2d(10, 15, 5)       # in = 63, out = 61
        self.conv4 = nn.Conv2d(15, 20, 7)

        self.pool = nn.MaxPool2d(3)

        # self.fc1 = nn.Linear(135, 64)
        self.o_n = nn.Linear(735, 1)


        self.flatten = nn.Flatten()
        self.activation = nn.ReLU()

    def forward(self, inpt):
        out = self.activation(self.conv1(inpt))
        out = self.pool(out)
        
        out = self.activation(self.conv2(out))
        out = self.pool(out)

        out = self.activation(self.conv3(out))
        out = self.pool(out)

        out = self.flatten(out)

        # print(out.shape)
        

        # out = self.activation(self.fc1(out))
        out = self.o_n(out)

        return out

In [45]:
class EarlyStopping:

    def __init__(self, patience=5, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)

        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [46]:
CNN().forward(torch.randn(1, 3, 256, 256))

tensor([[0.0812]], grad_fn=<AddmmBackward0>)

In [47]:
torchinfo.summary(CNN(), (5, 3, 256, 256))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [5, 1]                    14,720
├─Conv2d: 1-1                            [5, 5, 254, 254]          140
├─ReLU: 1-2                              [5, 5, 254, 254]          --
├─MaxPool2d: 1-3                         [5, 5, 84, 84]            --
├─Conv2d: 1-4                            [5, 10, 82, 82]           460
├─ReLU: 1-5                              [5, 10, 82, 82]           --
├─MaxPool2d: 1-6                         [5, 10, 27, 27]           --
├─Conv2d: 1-7                            [5, 15, 23, 23]           3,765
├─ReLU: 1-8                              [5, 15, 23, 23]           --
├─MaxPool2d: 1-9                         [5, 15, 7, 7]             --
├─Flatten: 1-10                          [5, 735]                  --
├─Linear: 1-11                           [5, 1]                    736
Total params: 19,821
Trainable params: 19,821
Non-trainable params: 0
Total

In [48]:
EPOCHS = 25

train_acc = torchmetrics.classification.BinaryAccuracy().to(device)
val_acc = torchmetrics.classification.BinaryAccuracy().to(device)

model = CNN().to(device)
optim = torch.optim.Adam(model.parameters(), lr = 3e-3)
criterion = torch.nn.BCELoss()
early_stopping = EarlyStopping(patience=3, verbose=True, path = 'mask_model.pth')


for epoch_num in range(EPOCHS):
    train_loss = 0
    i = 0

    bar = tqdm(train_dataloader)
    for img, label in bar:
        i+=1
        optim.zero_grad()

        img = img.to(device)
        label = label.to(device).unsqueeze(1)
        predictions = F.sigmoid(model(img))

        batch_loss = criterion(predictions, label)

        train_acc(predictions, label)
        batch_loss.backward()
        optim.step()

        train_loss+= batch_loss.item()
        bar.set_description_str("Training loss: {:.4f}, accuracy = {:.4f}".format(train_loss/i, train_acc.compute()))

    train_loss/=i

    
    with torch.no_grad():
        val_loss = 0
        i = 0
        bar = tqdm(val_dataloader)
        for img, label in bar:
            i+=1
            optim.zero_grad()

            img = img.to(device)
            label = label.to(device).unsqueeze(1)
            predictions = F.sigmoid(model(img))

            batch_loss = criterion(predictions, label)

            
            val_acc(predictions, label)
            val_loss+= batch_loss.item()
            bar.set_description_str("Validation loss: {:.4f}, accuracy = {:.4f}".format(val_loss/i, val_acc.compute()))

        val_loss/=i


    print("Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.4f}".format(epoch_num+1, EPOCHS, train_loss, train_acc.compute()))
    print("Epoch [{}/{}], Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(epoch_num+1, EPOCHS, val_loss, val_acc.compute()))
    early_stopping(val_loss, model)

    train_acc.reset()
    val_acc.reset()

    if early_stopping.early_stop:
        print("Early stopping")
        print('-'*60)
        break

Validation loss: 0.1588, accuracy = 0.9525: 100%|██████████| 4/4 [00:00<00:00,  8.78it/s]


Epoch [1/25], Train Loss: 0.3452, Train Accuracy: 0.8599
Epoch [1/25], Val Loss: 0.1588, Val Accuracy: 0.9525
Validation loss decreased (inf --> 0.158771).  Saving model ...


Validation loss: 0.1090, accuracy = 0.9613: 100%|██████████| 4/4 [00:00<00:00,  8.11it/s]


Epoch [2/25], Train Loss: 0.1822, Train Accuracy: 0.9320
Epoch [2/25], Val Loss: 0.1090, Val Accuracy: 0.9613
Validation loss decreased (0.158771 --> 0.108976).  Saving model ...


Validation loss: 0.0937, accuracy = 0.9712: 100%|██████████| 4/4 [00:00<00:00,  9.16it/s]


...

Validation loss: 0.0967, accuracy = 0.9750: 100%|██████████| 4/4 [00:00<00:00, 10.12it/s]


Epoch [6/25], Train Loss: 0.1458, Train Accuracy: 0.9474
Epoch [6/25], Val Loss: 0.0967, Val Accuracy: 0.9750
EarlyStopping counter: 2 out of 3


Validation loss: 0.0853, accuracy = 0.9750: 100%|██████████| 4/4 [00:00<00:00,  9.11it/s]

Epoch [7/25], Train Loss: 0.1364, Train Accuracy: 0.9527
Epoch [7/25], Val Loss: 0.0853, Val Accuracy: 0.9750
EarlyStopping counter: 3 out of 3
Early stopping
------------------------------------------------------------


: 